In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from jax import jit, grad
import qcsys as qs
import jax.numpy as jnp
from rar_helpers import *

In [3]:
import jaxquantum as jqt

In [4]:
# system, ϕ0, metrics0, system0 = get_system_normal_rar({})
ϕ0, metrics0, system0 = get_metrics_linear_rar({})
metrics0

{'g_ex': Array(-0.00212182+0.j, dtype=complex128),
 'ω_ATS': Array(6.02181564, dtype=float64),
 'ω_ResonatorA': Array(4.99584201, dtype=float64),
 'ω_ResonatorB': Array(7.00151547, dtype=float64)}

In [5]:
metrics, system, ϕ0, metrics0, system0 = get_metrics_normal_rar({})#{"ResonatorA_frequency": 3})
metrics["K_a"]*1e9

Array([ 0.00000000e+00, -3.55271368e-06,  3.55271368e-06, -3.55271368e-06,
       -1.06581410e-05,  2.13162821e-05], dtype=float64)

In [7]:
metrics, system, ϕ0, metrics0, system0 = get_metrics_normal_rar({
    "ATS__dE_J": 37.0 * 0.01
})
metrics["K_a"]*1e9

Array([-2.14061657e-01, -2.14015472e-01, -2.14154028e-01, -2.11546336e-01,
       -4.03521234e+01,  1.18006174e+05], dtype=float64)